In [ ]:
cd ~/setup
source ./.env.sh

#### 1. 시스템 환경설정

##### 1.1. 기본 환경


In [ ]:
# sudo vi /etc/apt/sources.list.d/ubuntu.sources
Types: deb
URIs: https://nexus.rnd6.ai-biz.net/repository/apt-repos/
Suites: noble noble-updates noble-backports
Components: main restricted universe multiverse
Signed-By: /usr/share/keyrings/ubuntu-archive-keyring.gpg

Types: deb
URIs: http://archive.ubuntu.com/ubuntu/
Suites: noble noble-updates noble-backports
Components: main restricted universe multiverse
Signed-By: /usr/share/keyrings/ubuntu-archive-keyring.gpg

Types: deb
URIs: http://security.ubuntu.com/ubuntu/
Suites: noble-security
Components: main restricted universe multiverse
Signed-By: /usr/share/keyrings/ubuntu-archive-keyring.gpg

In [ ]:
# sudo vi /etc/docker/daemon.json
{
    "ipv6": false,
    "insecure-registries": [
        "rnd6.ai-biz.net:5000",
        "rnd6.ai-biz.net:5001"
    ],
    "registry-mirrors": [
        "http://rnd6.ai-biz.net:5000"
    ]
}

In [ ]:
# sudo vi /etc/pip.conf
[global]
# 패키지를 다운로드할 때 기본적으로 사용하는 index-url
index-url = https://nexus.rnd6.ai-biz.net/repository/pypi-repos/simple
trusted-host = rnd6.ai-biz.net

# 타임아웃 설정 (초)
timeout = 60

##### 1.2. 환경변수

In [11]:
# 시스템 환경 (모든 사용자에 적용)
sudo cp ./assets/environment /etc/environment

##### 1.3. 환경설정

##### 1) 호스트명 변경

In [ ]:
# 호스트명 변경
sudo hostnamectl set-hostname rnd6

# 호스트명 변경(파일 수정 : rnd6)
sudo vi /etc/hosts

##### 2) DNS 변경

In [ ]:
# 인터페이스 DNS정보 변경 (nameservers 부분 2개의 IP 주소)
sudo vi /etc/netplan/50-cloud-init.yaml

# 이상하게 KT DNS는 결과가 안정적이지 않아서 GOOGLE로 변경함
dig trino.rnd6.ai-biz.net @8.8.8.8 +short

In [ ]:
network:
    ethernets:
        enp65s0f1:
          addresses:
          - 10.10.10.16/24
          nameservers:
            addresses:
            - 8.8.8.8
            - 8.8.4.4
            search: [ai-biz.net]
          routes:
          - to: default
            via: 10.10.10.1 
    version: 2

In [ ]:

# 변경적용
sudo netplan apply

# 상태확인
resolvectl status

# 캐시 비움
sudo resolvectl flush-caches

#### 2. Docker Compose 설치
- Docker : 27.4.0
- Docker Compose : 2.32.1

##### 2.1. Docker 설치

In [ ]:
# 패키지 관리자 업데이트
sudo apt-get update

Hit:1 https://packages.grafana.com/oss/deb stable InRelease
Hit:2 https://download.docker.com/linux/ubuntu noble InRelease            
Get:3 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]      
Hit:4 http://archive.ubuntu.com/ubuntu noble InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]        
Hit:6 http://apt.postgresql.org/pub/repos/apt noble-pgdg InRelease             
Get:7 http://security.ubuntu.com/ubuntu noble-security/main amd64 Components [7,240 B]
Get:8 http://archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:9 http://security.ubuntu.com/ubuntu noble-security/restricted amd64 Components [212 B]
Get:10 http://security.ubuntu.com/ubuntu noble-security/universe amd64 Components [52.0 kB]
Get:11 http://security.ubuntu.com/ubuntu noble-security/multiverse amd64 Components [212 B]
Get:12 http://archive.ubuntu.com/ubuntu noble-updates/main amd64 Packages [759 kB]
Get:13 http://archive.ubun

In [ ]:
sudo apt-get install -y ca-certificates curl gnupg

Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree... Done
Reading state information... 0% Reading state information... 0%Reading state information... Done
ca-certificates is already the newest version (20240203).
curl is already the newest version (8.5.0-2ubuntu10.6).
gnupg is already the newest version (2.4.4-2ubuntu17).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
# Docker GPG 키 추가
sudo install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --batch --yes --dearmor -o /etc/apt/keyrings/docker.gpg
sudo chmod a+r /etc/apt/keyrings/docker.gpg

In [ ]:
# Docker 리포지토리 설정
echo "deb [arch=`dpkg --print-architecture` signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu `lsb_release -cs` stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
cat /etc/apt/sources.list.d/docker.list

deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu noble stable


In [ ]:
# Docker 설치
sudo apt-get update

Hit:1 https://download.docker.com/linux/ubuntu noble InRelease
Hit:2 https://packages.grafana.com/oss/deb stable InRelease                    
Hit:3 http://apt.postgresql.org/pub/repos/apt noble-pgdg InRelease             
Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease               
Hit:5 http://archive.ubuntu.com/ubuntu noble InRelease
Hit:6 http://archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu noble-backports InRelease
Reading package lists... Done
W: https://packages.grafana.com/oss/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.


In [ ]:
sudo apt-get install -y docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
docker-ce is already the newest version (5:27.4.0-1~ubuntu.24.04~noble).
docker-ce-cli is already the newest version (5:27.4.0-1~ubuntu.24.04~noble).
containerd.io is already the newest version (1.7.24-1).
docker-buildx-plugin is already the newest version (0.19.2-1~ubuntu.24.04~noble).
docker-compose-plugin is already the newest version (2.31.0-1~ubuntu.24.04~noble).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.


In [ ]:
# Docker 설치 확인
docker --version

Docker version 27.4.0, build bde2b89


In [ ]:
# 그룹에 사용자 추가
sudo usermod -aG docker $(whoami)
# 권한 변경
sudo chown -R aibiz:aibiz /opt/containerd

##### 2.2. Docker Compose 설치

In [ ]:
# Compose 바이너리 다운로드
sudo curl -L "https://github.com/docker/compose/releases/latest/download/docker-compose-`uname -s`-`uname -m`" -o /usr/local/bin/docker-compose

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 61.6M  100 61.6M    0     0  30.7M      0  0:00:02  0:00:02 --:--:-- 94.1M


In [ ]:
# Compose 실행 가능하도록 설정
sudo chmod +x /usr/local/bin/docker-compose

In [ ]:
# Compose 설치 확인
docker-compose --version

Docker Compose version v2.32.1


In [ ]:
# 서비스 확인
sudo systemctl status docker --no-pager

● docker.service - Docker Application Container Engine
     Loaded: loaded (]8;;file://aibiz-ai-rnd6/usr/lib/systemd/system/docker.service/usr/lib/systemd/system/docker.service]8;;; enabled; preset: enabled)
     Active: active (running) since Wed 2024-12-18 14:01:29 KST; 1h 16min ago
TriggeredBy: ● docker.socket
       Docs: ]8;;https://docs.docker.comhttps://docs.docker.com]8;;
   Main PID: 1690404 (dockerd)
      Tasks: 28
     Memory: 27.1M (peak: 33.0M)
        CPU: 973ms
     CGroup: /system.slice/docker.service
             └─1690404 /usr/bin/dockerd -H fd:// --containerd=/run/containerd/c…

Dec 18 14:01:28 aibiz-ai-rnd6 dockerd[1690404]: time="2024-12-18T14:01:28.57…up"
Dec 18 14:01:28 aibiz-ai-rnd6 dockerd[1690404]: time="2024-12-18T14:01:28.57…nf"
Dec 18 14:01:28 aibiz-ai-rnd6 dockerd[1690404]: time="2024-12-18T14:01:28.67…t."
Dec 18 14:01:29 aibiz-ai-rnd6 dockerd[1690404]: time="2024-12-18T14:01:29.06…e."
Dec 18 14:01:29 aibiz-ai-rnd6 dockerd[1690404]: time="2024-12-

##### 2.3. Docker Compose 사용
- 공식 사이트 : https://docs.docker.com/manuals/


In [ ]:
# profile 사용
COMPOSE_PROFILES=debug docker compose down
docker compose --profile debug down

In [ ]:
# root 권한으로 접속
docker exec -i -t --user root <container_id> bash



#### 3. Python + Jupyter 설치
- Python     : 3.12.3  
  - Airflow(2.10.2) + SQLMesh(0.140.1) + MLflow(2.17.2) 
- JupyterHub : 5.2.1
- History    :
  - 문서작성중

##### 3.1. Python 설치

In [ ]:
# 시스템 환경에 Python(3.12.3) 설치
sudo apt update
sudo apt install python3 python3-pip python3-venv python-is-python3
sudo apt install libpq-dev python3-dev build-essential
python --version

In [ ]:
# 가상환경(airflow) 생성
sudo python3 -m venv /usr/local/venv/airflow
sudo chown -R aibiz:aibiz /usr/local/venv

# Airflow 설치(airflow-2.10.2)
source /usr/local/venv/airflow/bin/activate
pip install --upgrade pip setuptools wheel pandas
pip install apache-airflow[celery,postgres,redis]==2.10.2
airflow version

In [ ]:
# SQLMesh 설치(SQLMesh-0.140.1)
mkdir /opt/airflow/sqlmesh
mkdir /opt/airflow/sqlmesh/trino
source /usr/local/venv/airflow/bin/activate
pip install sqlmesh[web,slack]
pip install apache-airflow-providers-trino
pip install sqlmesh[trino,postgres]
sqlmesh  --version

In [ ]:
# MLflow 설치(mlflow-2.17.2, psycopg2-2.9.10)
source /usr/local/venv/airflow/bin/activate
pip install --upgrade pip
pip install mlflow psycopg2 boto3
mlflow --version

In [ ]:
# Jupyterhub 설치(5.2.1) -- 작업중
source /usr/local/venv/airflow/bin/activate
pip install --upgrade pip
pip install jupyterhub notebook jupyterlab
pip install bash_kernel
python3 -m bash_kernel.install --prefix=/usr/local/venv/jupyterhub
jupyter kernelspec list
jupyterhub --version

##### 3.2. Jupyterhub 설치

In [ ]:
# 가상환경(jupyter) 설정
sudo python3 -m venv /usr/local/venv/jupyterhub
sudo chown -R aibiz:aibiz /usr/local/venv


In [ ]:
# 셸 환경 설정(bash kernel 설치)
source /usr/local/venv/jupyterhub/bin/activate
pip install --upgrade pip
pip install jupyterhub notebook jupyterlab
pip install bash_kernel
python -m bash_kernel.install
jupyter kernelspec list
deactivate
# 버전(5.2.1) 확인
jupyterhub --version

In [ ]:
# 필수 패키지 설치
sudo apt-get update
sudo apt-get install -y curl nodejs npm

# JupyterHub 및 종속 패키지 설치
sudo npm install -g configurable-http-proxy

# 설정 파일 생성 및 편집
sudo mkdir -p /etc/jupyterhub
sudo chown $(whoami):$(whoami) /etc/jupyterhub
#jupyterhub --generate-config -f /etc/jupyterhub/jupyterhub_config.py
sudo cp ./jupyterhub/jupyterhub_config.py /etc/jupyterhub/.

# JupyterHub 실행 테스트
jupyterhub -f /etc/jupyterhub/jupyterhub_config.py

##### 3.3. 서비스 등록(필요시)

In [ ]:
# 서비스 등록
sudo cp ./jupyterhub/jupyterhub.service /etc/systemd/system/.

# 서비스 시작
sudo systemctl daemon-reload
sudo systemctl start jupyterhub
sudo systemctl status jupyterhub
sudo journalctl -u jupyterhub -f

# 시스템 부팅 시 자동 시작 설정
sudo systemctl enable jupyterhub